# Policy is valid under this setting

In [4]:
def target_compare(u_O, w_A = 1, w_O = 0.2, details = False):
    def one_threshold(threshold):
        print("Threshold:", threshold, "\n")
        target_policy = simu_target_policy_pattern(u_O = u_O, threshold = threshold)
        
        aa = []
        l = 5
        O = [[] for i in range(l**2)]
        D = [[] for i in range(l**2)]
        M  = [[] for i in range(l**2)]
        for seed in range(5):
            r = DG_once(seed = seed, l = 5, T = 14 * 48, time_dependent = False, w_A = w_A, w_O = w_O, sd_R  = 1e-5, sd_D = 3, u_O = u_O, 
                   TARGET = True, target_policy = target_policy, T_burn_in = 50,  dynamics = "old")
            a = np.mean(r[2][2])
            data = r[0]
            for i in range(l**2):
                data_i = data[i]
                O[i].append(np.mean([b[0][0] for b in data_i]))
                D[i].append(np.mean([b[0][1] for b in data_i]))
                M[i].append(np.mean([b[0][2] for b in data_i]))
            aa.append(a)
            
        if details:
            print(DASH, "means of Order:")
            for i in range(l):
                for j in range(l):
                    print(round(np.mean(O[i * l + j]),3), end = " ")
                print("\n")

            print(DASH, "means of Driver:")
            for i in range(l):
                for j in range(l):
                    print(round(np.mean(D[i * l + j]),3), end = " ")
                print("\n")

            print(DASH, "means of Mismatch:")
            for i in range(l):
                for j in range(l):
                    print(round(np.mean(M[i * l + j]),3), end = " ")
                print("\n")
        
        print("value:", np.mean(aa), np.std(aa))

        print(DASH)
    one_threshold(12) # a good policy
    one_threshold(120) # no reward
    a = DG_once(seed = 0, l = 5, T = 14 * 48, time_dependent = False, w_A = w_A, w_O = w_O, sd_R  = 1e-5, sd_D = 3, u_O = u_O, 
                   TARGET = False, target_policy = None, T_burn_in = 50, dynamics = "old")[2][2]
    print(np.mean(a)) # random reward

In [8]:
u_O = [[6 for j in range(5)] for i in range(5)]
u_O[1][1] = u_O[1][3] = u_O[3][1] = u_O[3][3] = 18
u_O[2][2] = 24
a = u_O[0]
for i in range(1,5):
    a += u_O[i]
u_O = a

# npseed(1)
# u_O = rlogN(2.5, .2, 5**2) 
target_compare(u_O, w_O = 0.05)

# print(DASH, "NEXT", DASH)

# u_O = [[10 for j in range(5)] for i in range(5)]
# u_O[1][1] = u_O[1][3] = u_O[3][1] = u_O[3][3] = 14
# u_O[2][2] = 18
# a = u_O[0]
# for i in range(1,5):
#     a += u_O[i]
# u_O = a

# # npseed(1)
# # u_O = rlogN(2.5, .2, 5**2) 
# target_compare(u_O)

Threshold: 12 

target policy: 

0 0 0 0 0 

0 1 0 1 0 

0 0 1 0 0 

0 1 0 1 0 

0 0 0 0 0 

means of Order: 

6 6 6 6 6 

6 18 6 18 6 

6 6 24 6 6 

6 18 6 18 6 

6 6 6 6 6 

value: 5.670819238727093 0.016559976076961625

--------------------------------------

Threshold: 120 

target policy: 

0 0 0 0 0 

0 0 0 0 0 

0 0 0 0 0 

0 0 0 0 0 

0 0 0 0 0 

means of Order: 

6 6 6 6 6 

6 18 6 18 6 

6 6 24 6 6 

6 18 6 18 6 

6 6 6 6 6 

value: 4.851261692732304 0.02180556159273956

--------------------------------------

4.619850845996925


# Simpler version of ours
1. both with or without M_t works well with QV

In [1]:
from _utility import *
from weight import *
from simu_funs import *
from main import *

In [9]:
def DG_once(seed = 1, T = 14 * 48 + 50, w_A = 8, w_O = 1, sd_R  = 1, sd_D = 1, 
            u_O = 12, p_behav = 0.5,
           TARGET = False, T_burn_in = 50):  
    """ prepare data
    """
    T = T + T_burn_in
    npseed(seed)
    w_M = 0.8 
    
    """ O: the pattern (spatial distribution) of orders
    """
#     O = rpoisson(u_O, (T))
    O = u_O + randn(T) / 10
    
    """ D: initial is the same with driver. then attract by the A and O. burn-in.
    """
    u_D = np.mean(u_O - 6)
    D = [u_D]
    
    """ Actions
    """
    if TARGET: # target. fixed. 
        A = rbin(1, 1, (T))
    else: # behaviour policy: random
        A = rbin(1, 0.5, (T))
    

    e_D = (rpoisson(1, (T)) - 1) * 0.01

    """ Initialization
    """
    M = [0.5] 
    R = []
    
    """ MAIN: state trasition and reward calculation [no action selection]
    """
    for t in range(1, T): 
        """ Drivers
        """
        ## attractions
        D_t = (u_D + D[t - 1] + w_A * A[t-1] ) / 2 
        
        D.append(D_t)
        """ New Order and Mismatch
        """
        O_t = O[t]
        M_t = w_M * (1 - abs(D_t - O_t) / abs(1 + D_t + O_t)) + (1 - w_M) * M[t - 1]
        M.append(M_t)
        
        """ Reward definitions
        """
#         R_t_1 = np.minimum(D_t, O_t)  
        R_t_1 = np.minimum(D_t, O_t) * M_t
        R.append(R_t_1)
    R.append(R_t_1) # add one more? # new reward?
    
    ## organization and burn-in; N * T
    R = arr(R).T[T_burn_in:]; D = arr(D).T[T_burn_in:]; M = arr(M).T[T_burn_in:]
    O = O[T_burn_in:]; A = A[T_burn_in:]
        
    """ reorganization
    """
    data_i = []
    for t in range(T - T_burn_in):
        data_i.append([arr([O[t], D[t], M[t]]), A[t], R[t]])
    return data_i, [[O, D, M], A, R]


In [10]:
def getRegionData(data_i):
    T = len(data_i) - 1
    tuples_i = []
    pi_Sit_1s = rbin(1, 1, (T)) # TARGET
    for t in range(T):
        tuple_t = data_i[t].copy()
        S_it1 = data_i[t + 1][0]
        A_it1 = data_i[t + 1][1]
        pi_Sit_1 = pi_Sit_1s[t]
        tuple_t += [S_it1, A_it1, pi_Sit_1]
        tuples_i.append(tuple_t)
    return tuples_i

In [11]:
def computeQV_one(tuples_i, R, penalty):
    
    ## prepare data
    R = arr([a[2] for a in tuples_i]) # began to use Rit
    T = len(tuples_i)
    mu, lam = penalty
    A_set = set([a[1] for a in tuples_i])
    
    ## get (S,A) pair

    Z = np.array([np.concatenate((a[0], [a[1]])) for a in tuples_i]) # T * p. [S, A]
    Zstar = np.array([np.concatenate((a[3], [a[5]])) for a in tuples_i])

    Z_tilde = np.vstack((Z, Zstar))
    
    ## kernel distance
    
    def SA_GRBF(Z, gamma):
        l = Z.shape[1] - 1
        T = Z.shape[0]
        A = Z[:, -1]
        A_mat = (A.reshape(-1,1) == A.reshape(1,-1))
        
        K = GRBF(Z[:,:(l-1)], Z[:,:(l-1)], gamma) + identity(T) * 1e-8
        return np.multiply(K, A_mat)
    
    gamma_g = 1 / (2 * np.median(pdist(Z[:,:(Z.shape[1]-1)]))**2)
    gamma_q = 1 / (2 * np.median(pdist(Z_tilde[:,:(Z_tilde.shape[1]-1)]))**2)
    Kg = SA_GRBF(Z, gamma_g)
    KQ = SA_GRBF(Z_tilde, gamma_q)
    # centeralization, p11
    ZTstar = np.mean(Z_tilde, 0)
    
    KQ = KQ - GRBF(Z_tilde, ZTstar.reshape(1, -1), gamma_q) -  GRBF(ZTstar.reshape(1, -1), Z_tilde, gamma_q) - GRBF(ZTstar.reshape(1, -1), ZTstar.reshape(1, -1), gamma_q)[0][0]
    
    ## Idnetity vec/mat
    C = np.hstack((-identity(T),identity(T)))       
    vec1, I = ones(T).reshape(-1,1), identity(T)
    
    E_right_bef_inverse = Kg + T * mu * I # RHS of E
    
    CKQ_1 = np.hstack((C.dot(KQ), -vec1))
    ECKQ1 = Kg.T.dot(solve(E_right_bef_inverse, CKQ_1)) # E[CK_Q,-1]
    
    left = (ECKQ1.T.dot(ECKQ1) + np.vstack((np.hstack((T * lam * KQ, zeros((T * 2, 1)))), zeros((1, T * 2 + 1))))) # Left part of (\hat{\alpha}, \hat{\eta})
    
    right = ECKQ1.T.dot(Kg.dot(solve(E_right_bef_inverse, R))) # Right part of (\hat{\alpha}, \hat{\eta})
    alpha_eta = -solve(left, np.expand_dims(right,1)) 
    alpha = alpha_eta[:(len(alpha_eta) - 1)]
    Vi = eta = alpha_eta[-1]
    
    """ no centeralization for KQ [? for below?]
    """

    Qvalues = alpha.T.dot(KQ)
    Qi_diff = Qvalues[0, T:] - Qvalues[0, :T] # Q^* - Q
         
    return Qi_diff, Vi

## NN works well!

In [11]:
def policy0(S):
    return 0.5
def policy1(S):
    return 1

# weight 里面的 retio 也需要改
# [S_it, A_it, R_it, Tsit, Tait, S_i(t+1), Tsi(t+1), pi_Sit_1, T_ait_1_pi]

# tuples_i[t] = [S_it, A_it, R_it, Tsit, Tait,  # 0 - 4
#                     S_i(t+1), Tsi(t+1), pi_Sit_1, T_ait_1_pi] # 5 - 8

def getWeight(tuples_i, policy0, policy1, dim_S_plus_Ts, 
              w_hidden = 10, Learning_rate = 1e-4,  n_layer = 2, 
              batch_size = 64, max_iteration = 1001, test_num = 0, epsilon = 1e-3,
             spatial = False, isValidation = False):
    # prepare transition pairs
    reg_weight = 0 # no penalty in the two-layer NN
    def concateOne(tuplet):
            return [tuplet[0], tuplet[1],
                    tuplet[3], tuplet[2]]
    SASR_i = [concateOne(tuplet) for tuplet in tuples_i]
    
    SASR_i = [SASR_i] # although we only need 1 layer of list
        
    obs_dim = int(dim_S_plus_Ts / 2)
    computeWeight = Density_Ratio_kernel(obs_dim = obs_dim, n_layer = n_layer, 
                                 w_hidden = w_hidden, Learning_rate = Learning_rate, reg_weight = reg_weight)
    
    weights = computeWeight.train(SASR_i, policy0, policy1, 
              batch_size = batch_size, max_iteration = max_iteration, n_neigh = None, 
              test_num = test_num, epsilon = epsilon, only_state = False, spatial = False)
    computeWeight.close_Session()
    
    return weights

In [13]:
values = []
for seed in range(10):
    data_i, details = DG_once(TARGET = False, seed = seed, T = 1000)
    tuples_i = getRegionData(data_i)
    R = details[2]
    R = R[:(len(R) - 1)]
    w = getWeight(tuples_i = tuples_i, policy0 = policy0, policy1 = policy1, dim_S_plus_Ts = 3 + 3)
    value = np.mean(w * R)
    print(value)
    values.append(value)
print(np.mean(values), np.std(values))

11.530979882392066
10.938511933279807
9.92381228219302
10.266099238194366
9.507713579235602
9.51745950205572
9.733662098366823
9.71867060213315
10.660901622605534
10.382218101797296
10.218002884225339 0.6351550865012732


In [12]:
V_MC = []
for i in range(10):
    V_MC.append(np.mean(DG_once(TARGET = True, seed = i)[1][2]))
print("TARGET:",np.mean(V_MC), np.std(V_MC))

V_MC = []
for i in range(10):
    V_MC.append(np.mean(DG_once(TARGET = False, seed = i)[1][2]))
print("Behav:",np.mean(V_MC), np.std(V_MC))

TARGET: 11.111894448616392 0.007121934698886889
Behav: 8.827027627284124 0.14494684990663562


## Susan's setting

In [1]:
from _utility import *
from weight import *
from simu_funs import *
from main import *

In [7]:
def DG_once_susan(seed = 1, T = 50 , N = 25, p = 0.5): 
    npseed(seed)
    for i in range(N):
        states = [randn(2)] 
        e1 = randn(T) / 4
        e2 = randn(T) / 4
        A = rbin(1, p, T).tolist()
        R = []
        for t in range(1,T):
            S1, S2 = states[t - 1]
            S11 = 3/4 * (2 * A[t-1] - 1 ) * S1 + 1 / 4 * S1 * S2 + e1[t]
            S22 = 3/4 * (1 - 2 * A[t-1] ) * S2 + 1 / 4 * S1 * S2 + e1[t]
            states.append([S11, S22])
            R.append(S11 + S22 / 2 + (2 * A[t-1]) / 4)
        R.append(0)
        if i == 0:
            As = A
            Rs = R
            statess = states
        else:
            As = As + A
            Rs = Rs + R
            statess = statess + states
    ## organization and burn-in; N * T
#     R = arr(R)
#     S = 
        
    """ reorganization
    """
    data_i = []
    for t in range((T - 1) * N):
        data_i.append([statess[t], As[t], Rs[t]])   
    return data_i, R

In [8]:
def getRegionData_susan(data_i, pi):
    T = len(data_i) - 1
    tuples_i = []
    pi_Sit_1s = rbin(1, pi, (T))
    for t in range(T):
        tuple_t = data_i[t].copy()
        S_it1 = data_i[t + 1][0]
        A_it1 = data_i[t + 1][1]
        pi_Sit_1 = pi_Sit_1s[t] # pi(S_{t+1})
        tuple_t += [S_it1, A_it1, pi_Sit_1]
        tuples_i.append(tuple_t)
    return tuples_i

In [11]:
def computeQV_one(tuples_i, R, penalty):
    
    ## prepare data
    R = arr([a[2] for a in tuples_i]) # began to use Rit
    T = len(tuples_i)
    mu, lam = penalty
    A_set = set([a[1] for a in tuples_i])
    
    ## get (S,A) pair

    Z = np.array([np.concatenate((a[0], [a[1]])) for a in tuples_i]) # T * p. [S, A]
    Zstar = np.array([np.concatenate((a[3], [a[5]])) for a in tuples_i])

    
    Z_tilde = np.vstack((Z, Zstar))
    
    ## kernel distance
    
    def SA_GRBF(Z, gamma):
        l = Z.shape[1] - 1
        T = Z.shape[0]
        A = Z[:, -1]
        A_mat = (A.reshape(-1,1) == A.reshape(1,-1))
        
        K = GRBF(Z[:,:(l-1)], Z[:,:(l-1)], gamma) + identity(T) * 1e-8
        return np.multiply(K, A_mat)
    
    gamma_g = 1 / (2 * np.median(pdist(Z[:,:(Z.shape[1]-1)]))**2)
    gamma_q = 1 / (2 * np.median(pdist(Z_tilde[:,:(Z_tilde.shape[1]-1)]))**2)
    Kg = SA_GRBF(Z, gamma_g)
    KQ = SA_GRBF(Z_tilde, gamma_q)
    # centeralization, p11
    ZTstar = np.mean(Z_tilde, 0)
    
    KQ = KQ - GRBF(Z_tilde, ZTstar.reshape(1, -1), gamma_q) -  GRBF(ZTstar.reshape(1, -1), Z_tilde, gamma_q) - GRBF(ZTstar.reshape(1, -1), ZTstar.reshape(1, -1), gamma_q)[0][0]
    
    ## Idnetity vec/mat
    C = np.hstack((-identity(T),identity(T)))       
    vec1, I = ones(T).reshape(-1,1), identity(T)
    
    E_right_bef_inverse = Kg + T * mu * I # RHS of E
    
    CKQ_1 = np.hstack((C.dot(KQ), -vec1))
    ECKQ1 = Kg.T.dot(solve(E_right_bef_inverse, CKQ_1)) # E[CK_Q,-1]
    
    left = (ECKQ1.T.dot(ECKQ1) + np.vstack((np.hstack((T * lam * KQ, zeros((T * 2, 1)))), zeros((1, T * 2 + 1))))) # Left part of (\hat{\alpha}, \hat{\eta})
    
    right = ECKQ1.T.dot(Kg.dot(solve(E_right_bef_inverse, R))) # Right part of (\hat{\alpha}, \hat{\eta})
    alpha_eta = -solve(left, np.expand_dims(right,1)) 
    alpha = alpha_eta[:(len(alpha_eta) - 1)]
    Vi = eta = alpha_eta[-1]
    
    """ no centeralization for KQ [? for below?]
    """

    Qvalues = alpha.T.dot(KQ)
    Qi_diff = Qvalues[0, T:] - Qvalues[0, :T] # Q^* - Q
         
    return Qi_diff, Vi

In [ ]:
values = []
for seed in range(3):
    data_i, R = DG_once_susan(p = 0.5, seed = seed, T = 100)# , N = 25
    tuples_i = getRegionData_susan(data_i, pi = 1)
    value = computeQV_one(tuples_i, R, penalty = [0.1, 0.1])[1]
    print(value)
    values.append(value)
print(np.mean(values), np.std(values))

In [13]:
V_MC = []
for seed in range(10):
    V_MC.append(np.mean(DG_once_susan(p = 0.5, seed = seed, T = 1000)[1]))
print("B:",np.mean(V_MC), np.std(V_MC))

V_MC = []
for seed in range(10):
    V_MC.append(np.mean(DG_once_susan(p = 1, seed = seed, T = 1000)[1]))
print("T1:",np.mean(V_MC), np.std(V_MC))


V_MC = []
for seed in range(10):
    V_MC.append(np.mean(DG_once_susan(p = 0, seed = seed, T = 1000)[1]))
print("T0:",np.mean(V_MC), np.std(V_MC))

B: 0.2655467109252993 0.013062885144231682
T1: 0.5453472021110254 0.02133186123333595
T0: 0.02890430693339857 0.01283215587099297
